In [9]:
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

In [5]:
words = open('names.txt', 'r').read().splitlines()
bigram_dict = {}
for word in words:
    chs = ['<S>'] + list(word) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        bigram_dict[bigram] = bigram_dict.get(bigram, 0) + 1

In [7]:
# Pre-process the data, creating a map of strings to ints to convert the input
# as well as an int to string mapping to convert the output

chars = sorted(list(set(''.join(words))))
stoi = { s: i+1 for i,s in enumerate(chars) }
stoi['.'] = 0
itos = { i: s for s,i in stoi.items() }

In [8]:
# Bigrams kinda suck
# Shall we use a neural network?
xs, ys = [], []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        rowIdx = stoi[ch1]
        colIdx = stoi[ch2]
        xs.append(rowIdx)
        ys.append(colIdx)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [42]:
# Forward pass
yenc = F.one_hot(ys, num_classes=27).float()
xenc = F.one_hot(xs, num_classes=27).float()

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True) # 27 features / 27 neurons
logits = xenc @ W
# Softmax
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(5), ys].log().mean()

In [43]:
W.grad = None
loss.backward()
loss.item()

3.7693049907684326

In [44]:
# Compute gradient, i.e. the influence of the weight on the loss function
W.grad[:3]

tensor([[ 0.0121,  0.0020,  0.0025,  0.0008,  0.0034, -0.1975,  0.0005,  0.0046,
          0.0027,  0.0063,  0.0016,  0.0056,  0.0018,  0.0016,  0.0100,  0.0476,
          0.0121,  0.0005,  0.0050,  0.0011,  0.0068,  0.0022,  0.0006,  0.0040,
          0.0024,  0.0307,  0.0292],
        [-0.1970,  0.0017,  0.0079,  0.0020,  0.0121,  0.0062,  0.0217,  0.0026,
          0.0025,  0.0010,  0.0205,  0.0017,  0.0198,  0.0022,  0.0046,  0.0041,
          0.0082,  0.0016,  0.0180,  0.0106,  0.0093,  0.0062,  0.0010,  0.0066,
          0.0131,  0.0101,  0.0018],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000]])

In [45]:
# Update the Weight tensor
W.data += -0.1 * W.grad

In [65]:
# Now it's time to refactor the code to make it concise 
# and actually train the network on the entire training set

In [81]:
# Model parameters
learning_rate = 50
num_iterations = 100

xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        rowIdx = stoi[ch1]
        colIdx = stoi[ch2]
        xs.append(rowIdx)
        ys.append(colIdx)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('Number of examples: ', num)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True) # 27 features / 27 neurons

Number of examples:  228146


In [ ]:
for k in range(num_iterations):
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # input to the network w/ one hot encoding
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # reset the gradient to 0
    loss.backward()
    
    # update
    W.data += -learning_rate * W.grad


3.758953809738159
3.3710994720458984
3.1540415287017822
3.020372152328491
2.9277102947235107
2.8604013919830322
2.809727907180786
2.77010178565979
2.738072395324707
2.711496353149414
2.6890029907226562
2.6696884632110596
2.6529300212860107
2.638277053833008
2.6253881454467773
